In [1]:
import pandas as pd
import urllib.parse
import brotli
import requests
from bs4 import BeautifulSoup

# import sk & tw politician test data

In [2]:
filein_path = "../../../data/data_wikis/1 - politician data for test scraping/"

In [3]:
sktest = pd.read_excel(filein_path+"sk_testset.xlsx")

In [4]:
sktest.shape

(10114, 23)

In [5]:
#twtest = pd.read_excel(filein_path+"tw_testset.xlsx")

In [6]:
#twtest.shape

# scrape namuwiki for sk politicians

In [7]:
test = sktest[0:2]

In [8]:
test

,Name_Native,Name_Chinese,Birthdate,Sex,Age,ElectionCategory_Native,ElectionDate,Position,L1_Division,L2_Division,...,CandidateNumber,CandidateStatus,WonElection,VoteCount,VotePercent,IncumbentStatus,Occupation,Education,Career1,Career2
0,가세로,賈世魯,19550815,남,58,구·시·군의 장선거,20140604,Chief,충청남도,태안군,...,4,등록,0,NaN,NaN,NaN,정당인,단국대학교 대학원 행정학박사,(전)박근혜대통령후보 대선 서산.태안총괄본부장,"(현)""박근혜를 사랑하는 모임"" 전국 상임고문"
1,가창현,賈昌鉉,19550315,남,59,구·시·군의회의원선거,20140604,Legislator,충청남도,태안군,...,7,등록,0,NaN,NaN,NaN,정치인,서산농림고등학교 졸업(현 서산중앙고등학교),(전)태안군환경산림과장,(전)태안군사회복지과장


In [9]:

url_stem = "https://namu.wiki/w/"

p_error_name = "data-v-1ac09f2e"
p_error_text = "해당 문서를 찾을 수 없습니다."

cookie_string = "__cfuid=grmF3qJrWaeC04yBkOj1xQ%3D%3D; __gads=ID=772102f7516dfc86:T=1655897825:S=ALNI_MZ3xD4yzier1J72mEv0mEaEQwvqEw; __gpi=UID=00000799dcfd5b64:T=1655897825:RT=1657615641:S=ALNI_Mbjd_NekMjY9txFw5RzlAsAyf-4sQ"
#__cf_bm=SmRl834WVJzDUb1wlAVgvmcp_dHu0rafA.rgr5rkekg-1658233984-0-AZU1FLWLC6Ljuol2MlRihdeiZQVpiVEKKPWAcBvEE5+13MlW4wG4E4ci1HJU32arjpGtRYAS0VdsnUWL3OF74x8="

cookies_dict = {"__cfuid":"grmF3qJrWaeC04yBkOj1xQ%3D%3D",
           "_gads":"ID=772102f7516dfc86:T=1655897825:S=ALNI_MZ3xD4yzier1J72mEv0mEaEQwvqEw",
           "_gpi":"UID=00000799dcfd5b64:T=1655897825:RT=1657615641:S=ALNI_Mbjd_NekMjY9txFw5RzlAsAyf-4sQ"}

In [14]:
# more complicated request header

request_headers = {"authority": "namu.wiki",
                   "method": "GET",
                   "path":"/w/%EA%B0%80%EC%84%B8%EB%A1%9C",
                   "scheme":"https",
                   "accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
                   "accept-encoding":"gzip, deflate, br",
                   "accept-language":"en-US,en;q=0.9,ko;q=0.8",
                   "cache-control":"max-age=0",
                   #"cookie":cookie_string,
                   "referer": "http://localhost:8888/", # this changed
                   "sec-ch-ua": '".Not/A)Brand";v="99", "Google Chrome";v="103", "Chromium";v="103"',
                   "sec-ch-ua-mobile":"?0",
                   "sec-ch-ua-platform":"Windows",
                   "sec-fetch-dest":"document",
                   "sec-fetch-mode":"navigate",
                   "sec-fetch-site":"same-origin",
                   "sec-fetch-user":"?1",
                   "upgrade-insecure-requests":"1",
                   "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}

In [27]:
# strategies

# 1. pass cookies? receive cookies?
# 2. block javascript?


# get namuwiki using requests
for index,p in test.iterrows():
    
    # construct url & request headers
    #url= url_stem+p.Name_Native
    url= url_stem+urllib.parse.quote(p.Name_Native.encode("utf-8"))
    #request_headers["path"]="/w/"+urllib.parse.quote(p.Name_Native.encode("utf-8"))
    
    print(p.Name_Native)
    print(urllib.parse.quote(p.Name_Native.encode("utf-8")))
    print(url)

    # request url
    #x = requests.get(url,headers=request_headers,cookies=cookies_dict)
    x = requests.get(url,cookies=cookies_dict)
    
    print(x.status_code)
    #print(x.headers)
    print(x.cookies)
    
    cookie_jar = x.cookies.get_dict()
    
    print(cookie_jar)  
    
    
    print(p.Name_Native)
    print(urllib.parse.quote(p.Name_Native.encode("utf-8")))
    print(url)
    
    #y = requests.get(url,headers=request_headers,cookies=cookie_jar)
    y = requests.get(url,cookies=cookie_jar)
        
    print(y.status_code)
    print(y.headers)
    print(y.cookies)

    
    if(True):
        
        # decode request object - which was coded using br (brotli algorithm)
        content = brotli.decompress(x.content)
    
        #print(content)

        # save output as an html file
        fout = open("test_"+p.Name_Native+".html","w")
        #fout.write(x.content.decode("utf-8"))
        fout.write(content.decode("utf-8"))
        fout.close()

        # convert to html using BeautifulSoup 

        soup = BeautifulSoup(content.decode("utf-8"),"html.parser")
        html = soup.prettify("utf-8")
        #print(html)

    


가세로
%EA%B0%80%EC%84%B8%EB%A1%9C
https://namu.wiki/w/%EA%B0%80%EC%84%B8%EB%A1%9C
403
<RequestsCookieJar[<Cookie __cf_bm=TOixWC.hsNbMEP3naAUsT9.twchUJmDuZdb71VPxY1s-1658243225-0-AVys5faIlAjfmZ5h0D8RqduoG2BGYTUw2KpJAthvF7RrRrkmk5Yaz76sa614nr4Hup/n1FgSV886Q73izM+oOZc= for .namu.wiki/>]>
{'__cf_bm': 'TOixWC.hsNbMEP3naAUsT9.twchUJmDuZdb71VPxY1s-1658243225-0-AVys5faIlAjfmZ5h0D8RqduoG2BGYTUw2KpJAthvF7RrRrkmk5Yaz76sa614nr4Hup/n1FgSV886Q73izM+oOZc='}
가세로
%EA%B0%80%EC%84%B8%EB%A1%9C
https://namu.wiki/w/%EA%B0%80%EC%84%B8%EB%A1%9C
403
{'Date': 'Tue, 19 Jul 2022 15:07:05 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'close', 'CF-Chl-Bypass': '1', 'Permissions-Policy': 'accelerometer=(),autoplay=(),camera=(),clipboard-read=(),clipboard-write=(),fullscreen=(),geolocation=(),gyroscope=(),hid=(),interest-cohort=(),magnetometer=(),microphone=(),payment=(),publickey-credentials-get=(),screen-wake-lock=(),serial=(),sync-xhr=(),usb=()', 'Cache-Control': 'pr

Error: Decompression error: b'RESERVED'

In [23]:
dict(x.headers)

{'Date': 'Tue, 19 Jul 2022 14:59:37 GMT',
 'Content-Type': 'text/html; charset=UTF-8',
 'Transfer-Encoding': 'chunked',
 'Connection': 'close',
 'CF-Chl-Bypass': '1',
 'Permissions-Policy': 'accelerometer=(),autoplay=(),camera=(),clipboard-read=(),clipboard-write=(),fullscreen=(),geolocation=(),gyroscope=(),hid=(),interest-cohort=(),magnetometer=(),microphone=(),payment=(),publickey-credentials-get=(),screen-wake-lock=(),serial=(),sync-xhr=(),usb=()',
 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0',
 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT',
 'X-Frame-Options': 'SAMEORIGIN',
 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"',
 'Set-Cookie': '__cf_bm=IYsthcWYxNAHK8oIIghIw7DglxQC2kvfjT74Hx1UFOM-1658242777-0-AV6EUTohhF28+M1f+hvrV1UStTtYW7VxU9pcO1wRdX9XBf5MVQNV8SwTK5dzVz79Mr6G7p74XaqFEew5oXTDrkU=; path=/; expires=Tue, 19-Jul-22 15:29:37 GMT; domain=.namu.wiki; HttpOnly; Secure; Sam